In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.datasets import fashion_mnist, mnist
from tensorflow.keras.backend import clear_session
from tensorflow.keras import backend as K
import scikitplot as skplt
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

plt.style.use('ggplot')

In [ ]:
# tf.config.list_physical_devices('GPU')

In [ ]:
layers_1_l = [
    Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
]

layers_2_l = [
    Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
]

layers_5_l = [
    Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
]

layers_10_l = [
    Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
]

layers_1_s = [
    Conv2D(4, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
]

layers_2_s = [
    Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
]

layers_5_s = [
    Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
]

layers_10_s = [
    Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(16, (3,3), padding='same', activation='relu'),
    Conv2D(12, (3,3), padding='same', activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
]

In [ ]:
cnns = {
    'CNN 1 S': layers_1_s,
    'CNN 2 S': layers_2_s,
    'CNN 5 S': layers_5_s,
    'CNN 10 S': layers_10_s,
    'CNN 1 L': layers_1_l,
    'CNN 2 L': layers_2_l,
    'CNN 5 L': layers_5_l,
    'CNN 10 L': layers_10_l,
}

In [ ]:
(X_train_org, y_train_org), (X_test_org, y_test_org) = fashion_mnist.load_data()

X_train_org = X_train_org.reshape(-1, 28, 28, 1)
X_test_org = X_test_org.reshape(-1, 28, 28, 1)

X_train_org = X_train_org.astype('float32')
X_test_org = X_test_org.astype('float32')
X_train_org /= 255
X_test_org /= 255

X_train_org, y_train_org = resample(X_train_org, y_train_org, random_state=0)
X_test_org, y_test_org = resample(X_test_org, y_test_org, random_state=0)

In [ ]:
def get_train_test_data(train_samples, test_samples):
    X_train = X_train_org[:train_samples]
    y_train = y_train_org[:train_samples]
    X_test = X_test_org[:test_samples]
    y_test = y_test_org[:test_samples]
    return X_train, y_train, X_test, y_test

In [ ]:
def calc_train_test_samples(total_samples, ratio):
    train_samples = int(total_samples * ratio)
    test_samples = total_samples - train_samples
    return train_samples, test_samples

In [ ]:
def plot_roc(y_test, y_pred):
    ax = skplt.metrics.plot_roc(y_test, y_pred, figsize=(8, 5))
    ax.set(xlim=(-0.04, 1.04), ylim=(-0.04, 1.04))
    return ax.figure

In [ ]:
def plot_cm(y_test, y_pred):
    ax = skplt.metrics.plot_confusion_matrix(y_test, y_pred.argmax(axis=1), normalize=True, figsize=(7, 7))
    return ax.figure

In [ ]:
def plot_pr(y_test, y_pred):
    ax = skplt.metrics.plot_precision_recall(y_test, y_pred, figsize=(8, 6))
    return ax.figure

In [ ]:
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
    
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1_m(y_true, y_pred):  
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def evaluate_svm(train_samples, test_samples):
    X_train, y_train, X_test, y_test = get_train_test_data(train_samples, test_samples)
    
    X_train = X_train.reshape(-1, 784)
    X_test = X_test.reshape(-1, 784)
    
    svc = SVC(C=5.0, kernel='rbf', probability=True)
    svc.fit(X_train, y_train)
    
    y_pred = svc.predict(X_test)
    y_pred_proba = svc.predict_proba(X_test)
    
    accuracy = cross_val_score(svc, X_train, y_train, cv=5).mean()
    val_accuracy = accuracy_score(y_test, y_pred)
    
    roc = plot_roc(y_test, y_pred_proba)
    pr = plot_pr(y_test, y_pred_proba)
    cm = plot_cm(y_test, y_pred_proba)
    
    return accuracy, val_accuracy, roc, pr, cm

In [ ]:
def evaluate_cnn(layers, train_samples, test_samples, epochs):
    X_train, y_train, X_test, y_test = get_train_test_data(train_samples, test_samples)    
    
    clear_session()
    np.random.seed(0x859)
    tf.random.set_seed(0x859)
    
    model = Sequential(layers)
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy', f1_m])

    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=True)
    
    loss = history.history['loss'][-1]
    accuracy = history.history['accuracy'][-1]
    f1 = history.history['f1_m'][-1]
    
    val_loss = history.history['val_loss'][-1]
    val_accuracy = history.history['val_accuracy'][-1]
    val_f1 = history.history['val_f1_m'][-1]
    
    # val_loss, val_accuracy, val_f1 = model.evaluate(X_test, y_test, verbose=False)
    
    y_pred_proba = model.predict(X_test, verbose=False)
    
    roc = plot_roc(y_test, y_pred_proba)
    pr = plot_pr(y_test, y_pred_proba)
    cm = plot_cm(y_test, y_pred_proba)
    
    return loss, accuracy, f1, val_loss, val_accuracy, val_f1, roc, pr, cm

In [ ]:
def select_size(results_df, size):
    return results_df[results_df['model'].str.endswith(size)]

In [ ]:
def plot_3a(data):
    ax = sns.lmplot(x='ratio', y='accuracy', hue='model', data=data)
    ax.set(xlim=(0.08, 0.92))
    return ax

In [ ]:
def plot_3b(data):
    ax = sns.lmplot(x='epochs', y='accuracy', hue='model', data=data)
    return ax

# 3a

In [ ]:
def gen_3a_svm():
    total_samples = 20000
    ratios = np.arange(0.025, 1.0, 0.025)
    
#     total_samples = 300
#     ratios = [0.2, 0.8]
    
    for ratio in ratios:
        train_samples, test_samples = calc_train_test_samples(total_samples, ratio)               
        accuracy, val_accuracy, roc, pr, cm = evaluate_svm(train_samples, test_samples)
        
        yield 'SVM', train_samples, test_samples, ratio, accuracy, val_accuracy        
        roc.savefig(f'plots/3a/svm_{ratio}_roc.svg')
        pr.savefig(f'plots/3a/svm_{ratio}_pr.svg')
        cm.savefig(f'plots/3a/svm_{ratio}_cm.svg')          
        plt.close('all')
        
        
results_3a_svm = pd.DataFrame(gen_3a_svm(), columns=[
    'model', 'train_samples', 'test_samples', 'ratio', 'accuracy', 'val_accuracy'
])
results_3a_svm.to_csv('results/3a_svm.csv')
results_3a_svm

In [ ]:
def gen_3a_cnn():
    total_samples = 20000
    epochs = 10
    ratios = np.arange(0.025, 1.0, 0.025)
    
#     total_samples = 300
#     epochs = 1
#     ratios = [0.2, 0.8]
    
    for ratio in ratios:
        train_samples, test_samples = calc_train_test_samples(total_samples, ratio)
        
        for name, layers in cnns.items():   
            loss, accuracy, f1, val_loss, val_accuracy, val_f1, roc, pr, cm = evaluate_cnn(layers, train_samples, test_samples, epochs)
            
            yield name, train_samples, test_samples, ratio, loss, accuracy, f1, val_loss, val_accuracy, val_f1
            
            codename = name.lower().replace(' ', '_')
            roc.savefig(f'plots/3a/{codename}_r{ratio}_roc.svg')
            pr.savefig(f'plots/3a/{codename}_r{ratio}_pr.svg')
            cm.savefig(f'plots/3a/{codename}_r{ratio}_cm.svg')             
            plt.close('all')
            
results_3a_cnn = pd.DataFrame(gen_3a_cnn(), columns=[    
    'model',
    'train_samples', 'test_samples', 'ratio',
    'loss', 'accuracy', 'f1', 'val_loss', 'val_accuracy', 'val_f1'
])
results_3a_cnn.to_csv('results/3a_cnn.csv')
results_3a_cnn

# 3b

In [ ]:
def gen_3b_cnn():
    total_samples = 60000
    epochses = range(1, 10+1)
    ratio = 0.8
    
#     total_samples = 2000
#     epochses = [1, 2]
    
    train_samples, test_samples = calc_train_test_samples(total_samples, ratio)
    
    for epochs in epochses:        
        for name, layers in cnns.items():   
            loss, accuracy, f1, val_loss, val_accuracy, val_f1, roc, pr, cm = evaluate_cnn(layers, train_samples, test_samples, epochs)
            
            yield name, train_samples, test_samples, epochs, loss, accuracy, f1, val_loss, val_accuracy, val_f1
            
            codename = name.lower().replace(' ', '_')            
            roc.savefig(f'plots/3b/{codename}_e{epochs}_roc.svg')
            pr.savefig(f'plots/3b/{codename}_e{epochs}_pr.svg')
            cm.savefig(f'plots/3b/{codename}_e{epochs}_cm.svg')            
            plt.close('all')
            
results_3b_cnn = pd.DataFrame(gen_3b_cnn(), columns=[    
    'model',
    'train_samples', 'test_samples', 'epochs',
    'loss', 'accuracy', 'f1', 'val_loss', 'val_accuracy', 'val_f1'
])
results_3b_cnn.to_csv('results/3b_cnn.csv')
results_3b_cnn

# Plots

In [ ]:
plot_3a(pd.concat((
    results_3a_svm,
    select_size(results_3a_cnn, 'S')
)))
plot_3a(pd.concat((
    results_3a_svm,
    select_size(results_3a_cnn, 'L')
)))
plot_3a(pd.concat((
    results_3a_svm,
    results_3a_cnn,
)))

In [ ]:
plot_3b(select_size(results_3b_cnn, 'S'))
plot_3b(select_size(results_3b_cnn, 'L'))
plot_3b(results_3b_cnn)